## Chapter 8: Storage and Blobs

In our e‑commerce application, we’ve built services for products, orders, and messaging. But every modern web application also needs to handle **files**—product images, user avatars, receipts, etc. Storing files in a database or on the local file system is not scalable or resilient. Instead, we use **blob storage**, a service optimized for storing large amounts of unstructured data.

Azure Blob Storage is a cloud‑scale object store that integrates seamlessly with .NET. With .NET Aspire, adding blob storage to your application is as simple as adding a component and defining a resource in the AppHost. In this chapter, you’ll learn how to:

- Model Azure Storage (blobs, queues, tables) in the AppHost.
- Use the `Aspire.Azure.Storage.Blobs` component to upload and download files.
- Run the Azurite emulator locally for development.
- Implement product image upload in your API and display images in the web frontend.
- Generate shared access signatures (SAS) for secure, time‑limited access.

By the end, your e‑commerce site will support product images stored in the cloud.

---

### 8.1 Why Blob Storage?

Traditional approaches to file storage include:

- **Database BLOB columns**: Stores files in the database. This increases database size, backup time, and can hurt performance.
- **Local file system**: Not scalable across multiple instances, and data can be lost if the instance is replaced.
- **Network attached storage (NAS)**: Complex and still a single point of failure.

Azure Blob Storage solves these problems by providing:

- **High durability and availability** (replicated across regions).
- **Scalability** to petabytes.
- **Access control** via shared access signatures (SAS) or Azure AD.
- **Integration with CDN** for fast global delivery.
- **Cost‑effectiveness** with tiered storage (hot, cool, archive).

In a microservices architecture, blob storage is often used as a central repository for files that multiple services need to access.

---

### 8.2 .NET Aspire Storage Components

Aspire provides components for Azure Storage services:

- **`Aspire.Azure.Storage.Blobs`**: for working with blob containers and blobs.
- **`Aspire.Azure.Storage.Queues`**: for working with Azure Queue Storage (a simple message queue).
- **`Aspire.Azure.Storage.Tables`**: for working with Azure Table Storage (NoSQL key‑value store).

These components register the appropriate SDK clients (`BlobServiceClient`, `QueueServiceClient`, `TableServiceClient`) and add health checks, logging, and distributed tracing.

For local development, you can use the **Azurite** emulator, which runs in a Docker container and simulates Azure Storage APIs. The Aspire hosting package makes it trivial to add Azurite to your AppHost.

---

### 8.3 Adding Azure Storage to the AppHost

First, install the hosting package for Azure Storage:

```bash
cd MyAspireApp.AppHost
dotnet add package Aspire.Hosting.Azure.Storage
```

Now in `Program.cs`, you can add an Azure Storage resource. To use the Azurite emulator locally, call `RunAsEmulator()`:

```csharp
var storage = builder.AddAzureStorage("storage")
    .RunAsEmulator();  // Runs Azurite container
```

From this storage resource, you can create child resources for blobs, queues, and tables:

```csharp
var blobs = storage.AddBlobs("productimages");
var queues = storage.AddQueues("orderimagesqueue");
```

Now reference these from your services. For example, the API service will need to upload images, so give it a reference to the blobs:

```csharp
var apiService = builder.AddProject<Projects.MyAspireApp_ApiService>("apiservice")
    .WithReference(blobs)   // injects connection string for blobs
    .WithReference(productsDb)
    .WithReference(messaging)
    .WithReference(appConfig);
```

The connection string will be injected as `ConnectionStrings__productimages`. If you’re using the emulator, the connection string points to the local Azurite instance.

If you want to use a real Azure Storage account in development, you can omit `RunAsEmulator()` and provide a connection string via a parameter:

```csharp
var storageConnectionString = builder.AddParameter("storage-connectionstring", secret: true);
var storage = builder.AddAzureStorage("storage")
    .WithConnectionString(storageConnectionString);
```

Then store the connection string in user secrets or environment variables.

---

### 8.4 Using the Blob Storage Component in the API Service

In the API service project, add the client component:

```bash
cd ../MyAspireApp.ApiService
dotnet add package Aspire.Azure.Storage.Blobs
```

In `Program.cs`, register the blob client:

```csharp
builder.AddAzureBlobClient("productimages");
```

This registers `BlobServiceClient` in DI, configured with the connection string from `ConnectionStrings:productimages`.

#### 8.4.1 Creating a Container

Before uploading blobs, you need a container. Typically, you’d create it on startup if it doesn’t exist. Add the following after `app.Build()` but before `app.Run()`:

```csharp
using Azure.Storage.Blobs;

var blobServiceClient = app.Services.GetRequiredService<BlobServiceClient>();
var containerClient = blobServiceClient.GetBlobContainerClient("product-images");
await containerClient.CreateIfNotExistsAsync();
```

This ensures the container exists. In production, you might create containers manually or via infrastructure as code, but for development this is convenient.

#### 8.4.2 Upload Endpoint

Now let’s create an endpoint to upload an image for a product. We’ll assume the product ID is provided and the image file is sent as `multipart/form-data`.

Add the necessary using statements and an endpoint in `Program.cs`:

```csharp
using Azure.Storage.Blobs;
using Azure.Storage.Blobs.Models;

app.MapPost("/products/{productId}/image", async (int productId, IFormFile file, BlobServiceClient blobServiceClient) =>
{
    if (file == null || file.Length == 0)
        return Results.BadRequest("No file uploaded.");

    // Allowed file types (e.g., images only)
    var allowedExtensions = new[] { ".jpg", ".jpeg", ".png", ".gif" };
    var extension = Path.GetExtension(file.FileName).ToLowerInvariant();
    if (!allowedExtensions.Contains(extension))
        return Results.BadRequest("Invalid file type.");

    // Generate a unique blob name (could include product ID and a GUID to avoid collisions)
    var blobName = $"{productId}/{Guid.NewGuid():N}{extension}";
    var containerClient = blobServiceClient.GetBlobContainerClient("product-images");
    var blobClient = containerClient.GetBlobClient(blobName);

    // Upload the file
    using var stream = file.OpenReadStream();
    await blobClient.UploadAsync(stream, new BlobUploadOptions
    {
        HttpHeaders = new BlobHttpHeaders { ContentType = file.ContentType }
    });

    // Store the blob URI in the product record (you'd update the product in the database)
    // For simplicity, we'll just return the URL.
    var blobUrl = blobClient.Uri.ToString();
    return Results.Ok(new { ImageUrl = blobUrl });
})
.WithName("UploadProductImage")
.Accepts<IFormFile>("multipart/form-data")
.Produces(200);
```

This endpoint:

- Validates the file presence and type.
- Generates a blob name that includes the product ID and a unique identifier.
- Uploads the file to the container with the appropriate content type.
- Returns the URL of the uploaded blob.

In a real application, you’d also update the product record in the database with this URL (or store it in a separate product‑images table). For now, we’ll just return the URL.

#### 8.4.3 Serving Images

The blob URL returned from Azure Storage (or Azurite) is publicly accessible if the container allows anonymous access. For security, you might want to keep containers private and generate SAS URLs. We’ll cover that later.

To test, you can run the app and use a tool like Postman or a simple HTML form to upload an image.

---

### 8.5 Displaying Images in the Web Frontend

In the web frontend, we need to display product images. First, let’s modify the product model to include an image URL. Since we don’t have a full product catalog yet, we’ll assume the product data comes from the API.

Update the `Product` class in the web frontend (or shared contracts) to have an `ImageUrl` property. Then, when fetching products, the API should include the image URL. For simplicity, we’ll just show an image after upload.

Create a simple Razor page in the web frontend to upload and display an image. In `Components/Pages/UploadImage.razor`:

```razor
@page "/upload-image/{productId:int}"
@inject HttpClient Http
@inject IConfiguration Config

<h3>Upload Image for Product @ProductId</h3>

<InputFile OnChange="OnFileSelected" />
<button @onclick="UploadFile" disabled="@(selectedFile == null)">Upload</button>

@if (!string.IsNullOrEmpty(imageUrl))
{
    <div>
        <h4>Uploaded Image:</h4>
        <img src="@imageUrl" style="max-width: 300px;" />
    </div>
}

@code {
    [Parameter] public int ProductId { get; set; }

    private IBrowserFile? selectedFile;
    private string? imageUrl;

    private void OnFileSelected(InputFileChangeEventArgs e)
    {
        selectedFile = e.File;
    }

    private async Task UploadFile()
    {
        if (selectedFile == null) return;

        using var content = new MultipartFormDataContent();
        using var fileStream = selectedFile.OpenReadStream(maxAllowedSize: 10 * 1024 * 1024); // 10 MB limit
        var streamContent = new StreamContent(fileStream);
        content.Add(streamContent, "file", selectedFile.Name);

        var response = await Http.PostAsync($"/products/{ProductId}/image", content);
        if (response.IsSuccessStatusCode)
        {
            var result = await response.Content.ReadFromJsonAsync<UploadResponse>();
            imageUrl = result?.ImageUrl;
        }
    }

    private class UploadResponse
    {
        public string ImageUrl { get; set; } = string.Empty;
    }
}
```

Add a navigation link to this page.

Now run the application, navigate to `/upload-image/1` (assuming product ID 1 exists), select an image, and upload. You should see the image displayed.

---

### 8.6 Using the Azurite Emulator

When you added `.RunAsEmulator()` in the AppHost, Aspire automatically starts an Azurite container. Azurite provides blob, queue, and table services on local ports. You can verify it's running by looking at the dashboard—you’ll see a resource named `storage` with child `productimages` (blobs). Clicking on the endpoint will show the Azurite URL (e.g., `http://localhost:10000/devstoreaccount1`).

Azurite persists data in a Docker volume by default, so blobs survive container restarts. If you want to reset, you can remove the volume.

The connection string injected into your service points to this emulator. It looks like:

```
DefaultEndpointsProtocol=http;AccountName=devstoreaccount1;AccountKey=Eby8vdM02xNOcqFlqUwJPLlmEtlCDXJ1OUzFT50uSRZ6IFsuFq2UVErCz4I6tq/K1SZFPTOtr/KBHBeksoGMGw==;BlobEndpoint=http://127.0.0.1:10000/devstoreaccount1;
```

This is automatically generated by Aspire.

---

### 8.7 Generating Shared Access Signatures (SAS)

In many scenarios, you don’t want your blobs to be publicly readable. Instead, you generate temporary, limited‑permission URLs called **Shared Access Signatures (SAS)**. For example, you might generate a URL that allows reading a specific blob for the next 5 minutes.

To generate a SAS token, you need access to the storage account key or use Azure AD authentication. With the `BlobServiceClient`, you can generate a SAS for a blob:

```csharp
public static string GenerateSasUri(BlobClient blobClient, DateTimeOffset expiresOn)
{
    // Create a user delegation key if using Azure AD, or use account key SAS.
    // Here we use account key SAS for simplicity.
    var sasBuilder = new BlobSasBuilder
    {
        BlobContainerName = blobClient.BlobContainerName,
        BlobName = blobClient.Name,
        Resource = "b", // b for blob
        ExpiresOn = expiresOn
    };
    sasBuilder.SetPermissions(BlobSasPermissions.Read);

    // Generate the SAS URI
    Uri sasUri = blobClient.GenerateSasUri(sasBuilder);
    return sasUri.ToString();
}
```

To use this, you need the storage account key. In the emulator, the key is well‑known; in production, you’d store it securely (e.g., in Key Vault). With Aspire, you can inject the connection string and extract the account key and name.

For a more secure approach, use **user delegation SAS** which requires Azure AD and does not expose the account key. But that’s beyond this chapter.

You can modify the upload endpoint to return a SAS URL instead of the public URL. But note: if the container is private, the public URL won't work; you must use SAS. So in a production setup, you'd typically return a SAS URL for immediate access, and perhaps store the blob path in the database.

---

### 8.8 Advanced: Image Processing with Queues

Often, after uploading an image, you want to perform processing—resize, generate thumbnails, extract metadata. This is a perfect job for a background worker. You can use Azure Queue Storage (or Service Bus) to trigger processing.

Add a queue to the AppHost:

```csharp
var queues = storage.AddQueues("imageprocessing");
```

Reference it from the API and a new worker service.

In the API, after uploading the blob, enqueue a message with the blob name and product ID:

```csharp
var queueClient = new QueueClient(storageConnectionString, "imageprocessing");
await queueClient.SendMessageAsync(JsonSerializer.Serialize(new { BlobName = blobName, ProductId = productId }));
```

Then create a worker service that listens to the queue, processes the image (e.g., using SixLabors.ImageSharp), and uploads a thumbnail to another container.

This demonstrates the power of combining blob storage with messaging.

---

### 8.9 Hands‑on: Add Product Images to Your E‑Commerce App

Let's extend our application with a complete product image feature.

**Step 1: Update the AppHost** as shown above (add storage with emulator, blobs, and optionally queues).

**Step 2: Add the blob component to ApiService** and create the upload endpoint.

**Step 3: Add a simple product list page in the web frontend** that shows product images. For that, you'll need to modify the API to return product data including image URL. Store the image URL in the database when uploading.

To keep the database updated, after successful upload, update the product record:

```csharp
// Inside the upload endpoint, after upload, update the product
using var dbConnection = await dataSource.OpenConnectionAsync();
await dbConnection.ExecuteAsync(
    "UPDATE Products SET ImageUrl = @ImageUrl WHERE Id = @ProductId",
    new { ImageUrl = blobUrl, ProductId = productId });
```

Now when fetching products, the API can return the image URL.

**Step 4: Create a product list page** in the web frontend that calls `GET /products` and displays images.

**Step 5: (Optional) Add a thumbnail generation worker** using queues and the `Aspose.Azure.Storage.Queues` component.

---

### 8.10 Summary

In this chapter, you learned how to integrate Azure Blob Storage into your Aspire application:

- **Azure Storage hosting packages** let you add storage resources and run Azurite locally.
- **Blob components** register `BlobServiceClient` and add health checks, telemetry.
- You implemented file upload and download, using both public and SAS URLs.
- You saw how to combine blobs with queues for background processing.

With product images, your e‑commerce site becomes much more realistic. In the next chapter, we’ll dive into **Advanced Orchestration and Patterns**, exploring custom resources, lifecycle hooks, and more sophisticated service discovery. You’ll learn how to extend Aspire’s orchestration to meet complex requirements.

---

**Exercises**

1. Modify the upload endpoint to generate a SAS URL valid for 24 hours and return that instead of the public URL. Test that the SAS URL works when the container is private.
2. Add a second blob container for storing product thumbnails. Create a background worker (using a new project) that listens to a queue and generates a thumbnail for each uploaded image, then stores it in the thumbnails container.
3. Explore the Azurite emulator’s data volume. Locate the volume used by Aspire and inspect the stored blobs.
4. In the web frontend, add an image gallery for a product that shows multiple images (modify the database to allow multiple images per product).

In Chapter 9, we’ll explore **Advanced Orchestration and Patterns** to take your Aspire skills to the next level.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='7. externalizing_configuration_with_aspire.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../3. advanced_orchestration_and_patterns/9. custom_resources_and_lifecycle_hooks.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
